In [3]:
import os
import logging

from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, IntegerType, DateType
from confluent_kafka.serialization import StringDeserializer
from confluent_kafka.schema_registry import SchemaRegistryClient as con
from confluent_kafka.serialization import StringSerializer
from schema_registry.client import SchemaRegistryClient, schema
from schema_registry.serializers import JsonMessageSerializer

In [4]:
# Настройка логирования
logging.getLogger().setLevel(logging.INFO)
logger = logging.getLogger("main")

In [5]:
KAFKA_BOOTSTRAP_SERVERS = "kafka111-mirror:9192" # kafka111-mirror:9192 localhost:9095
SR_URL = os.getenv("SCHEMA_REGISTRY_URL", "http://localhost:8081/") # localhost:8081 schema-registry:8081
HDFS_URI = "http://hadoop-namenode:9870" # "http://hadoop-namenode:9870"
KAFKA_ORDERS_TOPIC = os.getenv("KAFKA_ORDERS_TOPIC", "orders")

In [6]:
SR_client = SchemaRegistryClient(url=SR_URL) #сторонний клиент
#schema_registry_client = SchemaRegistryClient({'url': SR_URL})
json_deserializer = JsonMessageSerializer(SR_client)
string_deserializer = StringDeserializer('utf_8')

In [7]:
subject = f'{KAFKA_ORDERS_TOPIC}-value'
confluent_SR_client = con({"url": SR_URL})
latest = confluent_SR_client.get_latest_version(subject)
schema_file = latest.schema.schema_str
json_value_schema_str = schema.JsonSchema(schema_file)

In [8]:
# Спарк сессия
spark = SparkSession.builder \
        .appName("SparkApp") \
        .config("spark.executor.memory", "2g") \
        .config("spark.executor.cores", "1") \
        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1") \
        .config("spark.hadoop.fs.defaultFS", "hdfs://hadoop-namenode:9000") \
        .config("spark.hadoop.dfs.namenode.http-address", "hadoop-namenode:9870") \
        .config("spark.hadoop.dfs.namenode.kerberos.principal", "dummy") \
        .config("spark.hadoop.mapreduce.job.user.name", "root") \
        .config("spark.hadoop.user.name", "root") \
        .config("spark.driver.extraJavaOptions", "-Divy.home=/tmp/ivy") \
        .config("spark.sql.catalogImplementation", "in-memory") \
        .config("spark.hadoop.fs.hdfs.impl", "org.apache.hadoop.hdfs.DistributedFileSystem") \
        .config("spark.hadoop.fs.file.impl", "org.apache.hadoop.fs.LocalFileSystem") \
        .getOrCreate()
logger.info("Spark session started.")

In [9]:
orders_path = "hdfs://hadoop-namenode:9000/data/orders"

In [10]:
# Вычитываем заказы
df = spark.read.option("multiline", "false").json(orders_path)

In [11]:
df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- email: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- brand: string (nullable = true)
 |    |    |-- category: string (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- index: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- product_id: string (nullable = true)
 |    |    |-- sku: string (nullable = true)
 |    |    |-- store_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- state: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [12]:
df.show(5)

+-----------+--------------------+--------------------+-------------------+--------+---------+-------------------+
|customer_id|               email|               items|         order_date|order_id|    state|          timestamp|
+-----------+--------------------+--------------------+-------------------+--------+---------+-------------------+
|        Bob|brittneywilliams@...|[{EFG, Мобильные ...|2025-07-09 08:36:46|  270184|Cancelled|2025-08-09T20:55:53|
|        Bob|  alex07@example.net|[{PQR, Аудио, Пор...|2025-03-19 08:57:10|  265848|Completed|2025-08-09T20:59:51|
|        Bob|  hscott@example.net|[{EFG, Мобильные ...|2025-05-23 09:13:31|  268416|Cancelled|2025-08-09T21:03:56|
|        Bob|  todd77@example.net|[{JKL, Аудио, Бес...|2024-11-04 21:36:30|  272106|Completed|2025-08-09T21:04:57|
|        Bob|lynnalvarez@examp...|[{MNO, Электроник...|2025-02-28 11:50:55|  268029|Cancelled|2025-08-09T21:09:03|
+-----------+--------------------+--------------------+-------------------+-----

In [13]:
# Разворачиваем items массив
df2 = df.withColumn("item", F.explode(F.col("items")))
df2.show(10) #, truncate=False

+-----------+--------------------+--------------------+-------------------+--------+---------+-------------------+--------------------+
|customer_id|               email|               items|         order_date|order_id|    state|          timestamp|                item|
+-----------+--------------------+--------------------+-------------------+--------+---------+-------------------+--------------------+
|        Bob|brittneywilliams@...|[{EFG, Мобильные ...|2025-07-09 08:36:46|  270184|Cancelled|2025-08-09T20:55:53|{EFG, Мобильные т...|
|        Bob|brittneywilliams@...|[{EFG, Мобильные ...|2025-07-09 08:36:46|  270184|Cancelled|2025-08-09T20:55:53|{STU, Электроника...|
|        Bob|brittneywilliams@...|[{EFG, Мобильные ...|2025-07-09 08:36:46|  270184|Cancelled|2025-08-09T20:55:53|{BCD, Хранение да...|
|        Bob|brittneywilliams@...|[{EFG, Мобильные ...|2025-07-09 08:36:46|  270184|Cancelled|2025-08-09T20:55:53|{VWX, Компьютеры,...|
|        Bob|brittneywilliams@...|[{EFG, Мобильн

In [14]:
# Более лучше разворачиваем
exploded_df2 = df2.withColumn("item", F.explode(F.col("items"))).select("*", "item.*").drop("item", "items")
exploded_df2.show(10)

+-----------+--------------------+-------------------+--------+---------+-------------------+-----+------------------+--------------------+--------+--------------------+----------+---------+---------+
|customer_id|               email|         order_date|order_id|    state|          timestamp|brand|          category|         description|   index|                name|product_id|      sku| store_id|
+-----------+--------------------+-------------------+--------+---------+-------------------+-----+------------------+--------------------+--------+--------------------+----------+---------+---------+
|        Bob|brittneywilliams@...|2025-07-09 08:36:46|  270184|Cancelled|2025-08-09T20:55:53|  EFG|Мобильные телефоны|Смартфон с камеро...|products|Мобильный телефон...|     12356|EFG-12356|store_001|
|        Bob|brittneywilliams@...|2025-07-09 08:36:46|  270184|Cancelled|2025-08-09T20:55:53|  STU|       Электроника|Электронная книга...|products|Электронная книга...|     12352|STU-12352|store_

In [15]:
# временные ренеймы
pre_analytics_df = exploded_df2.withColumnRenamed("timestamp", "timestamp_str").withColumnRenamed("product_id", "product_id_str")
pre_analytics_df.show(10)

+-----------+--------------------+-------------------+--------+---------+-------------------+-----+------------------+--------------------+--------+--------------------+--------------+---------+---------+
|customer_id|               email|         order_date|order_id|    state|      timestamp_str|brand|          category|         description|   index|                name|product_id_str|      sku| store_id|
+-----------+--------------------+-------------------+--------+---------+-------------------+-----+------------------+--------------------+--------+--------------------+--------------+---------+---------+
|        Bob|brittneywilliams@...|2025-07-09 08:36:46|  270184|Cancelled|2025-08-09T20:55:53|  EFG|Мобильные телефоны|Смартфон с камеро...|products|Мобильный телефон...|         12356|EFG-12356|store_001|
|        Bob|brittneywilliams@...|2025-07-09 08:36:46|  270184|Cancelled|2025-08-09T20:55:53|  STU|       Электроника|Электронная книга...|products|Электронная книга...|         12

In [16]:
pre_analytics_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- email: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- state: string (nullable = true)
 |-- timestamp_str: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- category: string (nullable = true)
 |-- description: string (nullable = true)
 |-- index: string (nullable = true)
 |-- name: string (nullable = true)
 |-- product_id_str: string (nullable = true)
 |-- sku: string (nullable = true)
 |-- store_id: string (nullable = true)



In [17]:
# Подготовка к аналитике
pre_analytics_df2 = pre_analytics_df.withColumn("timestamp", F.to_timestamp(F.col("timestamp_str"))) \
                                    .withColumn("product_id", F.col("product_id_str").cast(IntegerType())) \
                                    .drop("timestamp_str") \
                                    .drop("product_id_str")
pre_analytics_df2.show(10)

+-----------+--------------------+-------------------+--------+---------+-----+------------------+--------------------+--------+--------------------+---------+---------+-------------------+----------+
|customer_id|               email|         order_date|order_id|    state|brand|          category|         description|   index|                name|      sku| store_id|          timestamp|product_id|
+-----------+--------------------+-------------------+--------+---------+-----+------------------+--------------------+--------+--------------------+---------+---------+-------------------+----------+
|        Bob|brittneywilliams@...|2025-07-09 08:36:46|  270184|Cancelled|  EFG|Мобильные телефоны|Смартфон с камеро...|products|Мобильный телефон...|EFG-12356|store_001|2025-08-09 20:55:53|     12356|
|        Bob|brittneywilliams@...|2025-07-09 08:36:46|  270184|Cancelled|  STU|       Электроника|Электронная книга...|products|Электронная книга...|STU-12352|store_002|2025-08-09 20:55:53|     12

In [18]:
pre_analytics_df2.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- email: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- state: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- category: string (nullable = true)
 |-- description: string (nullable = true)
 |-- index: string (nullable = true)
 |-- name: string (nullable = true)
 |-- sku: string (nullable = true)
 |-- store_id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- product_id: integer (nullable = true)



In [19]:
# Топ 5 покупателей по личеству купленных товаров
analytics_df = pre_analytics_df.groupBy("customer_id") \
        .agg(F.count("*").alias("items_purchased")) \
        .orderBy(F.col("items_purchased").desc())
analytics_df.show(5)

+-----------+---------------+
|customer_id|items_purchased|
+-----------+---------------+
|        Bob|            866|
|        Eva|            515|
|    Charlie|            508|
|      David|            460|
|      Alice|            442|
+-----------+---------------+



In [20]:
analytics_schema = StructType([
    StructField("customer_id", StringType(), nullable=True),
    StructField("items_purchased", IntegerType(), nullable=True)])

In [21]:
empty_analytics_df = spark.createDataFrame([], schema=analytics_schema)
union_df=empty_analytics_df.union(analytics_df)
union_df2 = union_df.withColumn("items_purchased", F.col("items_purchased").cast(IntegerType()))
union_df2.show(5)
union_df2.printSchema()

+-----------+---------------+
|customer_id|items_purchased|
+-----------+---------------+
|        Bob|            866|
|        Eva|            515|
|    Charlie|            508|
|      David|            460|
|      Alice|            442|
+-----------+---------------+

root
 |-- customer_id: string (nullable = true)
 |-- items_purchased: integer (nullable = true)



In [22]:
# schema-registry:8081
subject_a = 'analytic-topic-value'
confluent_SR_client = con({"url": SR_URL})
latest_a = confluent_SR_client.get_latest_version(subject_a)
schema_file_a = latest_a.schema.schema_str
json_value_schema_a_str = schema.JsonSchema(schema_file_a)
print(json_value_schema_a_str)

{'$schema': 'http://json-schema.org/draft-07/schema#', 'title': 'AnalisMessageSchema', 'type': 'object', 'properties': {'customer_id': {'type': 'string'}, 'items_purchased': {'type': 'integer'}}, 'required': ['customer_id', 'items_purchased']}


In [23]:
# Записываем аналитику в Kafka
analytics_df.selectExpr("to_json(struct(*)) AS value") \
    .write \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka111-mirror:9192") \
    .option("topic", "analytic-topic") \
    .option("value.serializer", "io.confluent.connect.json.JsonSchemaConverter") \
    .option("schema.registry.url", os.getenv("KAFKA_SCHEMA_REGISTRY", "localhost:8081")) \
    .save()

In [24]:
r_df = pre_analytics_df2.withColumnRenamed("customer_id", "user_id").withColumnRenamed("product_id", "product_id_tmp")
r_df.show(10)
r_df.printSchema()

+-------+--------------------+-------------------+--------+---------+-----+------------------+--------------------+--------+--------------------+---------+---------+-------------------+--------------+
|user_id|               email|         order_date|order_id|    state|brand|          category|         description|   index|                name|      sku| store_id|          timestamp|product_id_tmp|
+-------+--------------------+-------------------+--------+---------+-----+------------------+--------------------+--------+--------------------+---------+---------+-------------------+--------------+
|    Bob|brittneywilliams@...|2025-07-09 08:36:46|  270184|Cancelled|  EFG|Мобильные телефоны|Смартфон с камеро...|products|Мобильный телефон...|EFG-12356|store_001|2025-08-09 20:55:53|         12356|
|    Bob|brittneywilliams@...|2025-07-09 08:36:46|  270184|Cancelled|  STU|       Электроника|Электронная книга...|products|Электронная книга...|STU-12352|store_002|2025-08-09 20:55:53|         12

In [25]:
r_df2 = r_df.select("user_id", "product_id_tmp")
r_df2.show(5, truncate=False)
r_df2.printSchema()

+-------+--------------+
|user_id|product_id_tmp|
+-------+--------------+
|Bob    |12356         |
|Bob    |12352         |
|Bob    |12355         |
|Bob    |12353         |
|Bob    |12349         |
+-------+--------------+
only showing top 5 rows

root
 |-- user_id: string (nullable = true)
 |-- product_id_tmp: integer (nullable = true)



In [26]:
r_df3= r_df2.groupBy("user_id", "product_id_tmp") \
                .agg(
                    F.count("*").alias("product_cnt"))
r_df3.show(5,truncate=False)

+-------+--------------+-----------+
|user_id|product_id_tmp|product_cnt|
+-------+--------------+-----------+
|Bob    |12354         |83         |
|Bob    |12355         |92         |
|Bob    |12352         |90         |
|Bob    |12356         |97         |
|Bob    |12349         |94         |
+-------+--------------+-----------+
only showing top 5 rows



In [27]:
w = Window.partitionBy("user_id").orderBy(F.desc("product_cnt"), F.asc("product_id_tmp"))

In [28]:
top3_per_user = r_df3.withColumn("rn", F.row_number().over(w)) \
                      .filter(F.col("rn") <= 3) \
                      .drop("rn") \
                      .orderBy("user_id", F.desc("product_cnt"))
top3_per_user.show()

+-------+--------------+-----------+
|user_id|product_id_tmp|product_cnt|
+-------+--------------+-----------+
|  Alice|         12355|         65|
|  Alice|         12351|         63|
|  Alice|         12350|         58|
|    Bob|         12353|        116|
|    Bob|         12350|        107|
|    Bob|         12357|         98|
|Charlie|         12351|         69|
|Charlie|         12357|         68|
|Charlie|         12352|         61|
|  David|         12350|         66|
|  David|         12351|         58|
|  David|         12355|         53|
|    Eva|         12352|         70|
|    Eva|         12357|         68|
|    Eva|         12349|         66|
+-------+--------------+-----------+



In [29]:
# Тов в список рекомендуемых product_id 
agg_df = top3_per_user.groupBy("user_id") \
                   .agg(F.collect_list(F.struct(F.col("product_cnt"), F.col("product_id_tmp"))).alias("items"))

agg_df2 = agg_df.withColumn("items_sorted", F.reverse(F.array_sort(F.col("items")))) \
            .withColumn("product_id", F.expr("transform(items_sorted, x -> x.product_id_tmp)")) \
            .select("user_id", "product_id")

agg_df2.show(truncate=False)

+-------+---------------------+
|user_id|product_id           |
+-------+---------------------+
|Eva    |[12352, 12357, 12349]|
|Charlie|[12351, 12357, 12352]|
|Bob    |[12353, 12350, 12357]|
|Alice  |[12355, 12351, 12350]|
|David  |[12350, 12351, 12355]|
+-------+---------------------+



In [32]:
rec_schema = StructType([
    StructField("user_id", StringType(), nullable=True),
    StructField("product_id", IntegerType(), nullable=True)])

In [33]:
out_df = top3_per_user.withColumnRenamed("product_id_tmp", "product_id").select("user_id", "product_id")
out_df.show(truncate=False)
out_df.printSchema()

+-------+----------+
|user_id|product_id|
+-------+----------+
|Alice  |12355     |
|Alice  |12351     |
|Alice  |12350     |
|Bob    |12353     |
|Bob    |12350     |
|Bob    |12357     |
|Charlie|12351     |
|Charlie|12357     |
|Charlie|12352     |
|David  |12350     |
|David  |12351     |
|David  |12355     |
|Eva    |12352     |
|Eva    |12357     |
|Eva    |12349     |
+-------+----------+

root
 |-- user_id: string (nullable = true)
 |-- product_id: integer (nullable = true)



In [34]:
# Записываем рекомендации в Kafka
out_df.selectExpr("to_json(struct(*)) AS value") \
    .write \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka111:9092") \
    .option("topic", "recommendations") \
    .save()